## Reactive Functions & Expressions

In the [Dependencies and Watchers](Dependencies_and_Watchers.ipynb) guide we discovered how to express dependencies and write callbacks that are invoked when parameter values change. This imperative style of expressing dynamic behavior is powerful but can also end up resulting in complex code that is hard to read and maintain. Therefore Param 2.0 introduces a set of primitives that make it possible to express complex dependencies in a declarative form resulting in reactive functions and expressions that are re-evaluated automatically when required.

The reactive programming model is one you might be familiar with from Excel, where formulas can reference cells or ranges and dynamically (or rather reactively) recompute when the inputs to a formula changes. In Param, Parameter objects represent the inputs or references in a formula and reactive expressions replace the concept of a formula. Additionally `param.bind` allows the creation of a reactive function with arbitrary inputs.

This user guide is structured as two main sections:

- [Reactive Functions](#Reactive-Functions): Using `param.bind` to declare functions that react when their inputs change.
- [Reactive Expressions](#Reactive-Expressions): Using `param.reactive` to wrap ordinary objects and/or parameters in a proxy that acts like the underlying object but reacts when an input changes.

## Reactive Functions

Let us start by creating a `Parameterized` class with some parameters, here named `a` and `b`. Additionally we import `param.ipython` to allow reactive components to display correctly and re-render themselves when the inputs change.

In [ ]:
import param
import param.ipython

class Parameters(param.Parameterized):
    
    a = param.Number(default=1)

    b = param.Number()

    run = param.Event()

Now we will create a reactive function using `param.bind`. We start by declaring a function that adds two parameters and then bind parameters `a` and `b` to that function:

In [ ]:
def add(a, b):
    return a + b

p = Parameters()

reactive_add = param.bind(add, p.param.a, p.param.b)

reactive_add

Binding parameters to a function returns a new bound function which will behave like `add` called with the given positional and keyword arguments. You can see that this bound function renders the output and when we update one of the inputs to the function the output will reflect the updated result.

In [ ]:
p.a = 5

reactive_add

We can also call the reactive function explicitly to return the current result:

In [ ]:
reactive_add()

Note that if you only partially bind the required arguments the function will not be automatically evaluated:

In [ ]:
param.bind(add, p.param.a)

## Reactive Expressions

While reactive functions are very useful and allow writing arbitrary logic they still require writing the actual functions which results in more indirection and less readability. With a `reactive` expression you can wrap any object or parameter value and apply operations on it, as if you are working with the actual object. In other words, it acts as a proxy of the underlying value and supports (almost) all operations that can be performed with the original object.

In order to avoid any clashes between the namespace of the reactive expression and the object it is wrapping, it has a special name space called `.rx`, that contains special methods to work with the expression.


### Using Parameters

As an example we can create reactive proxies for the `a` and `b` parameters and add them together:

In [ ]:
p = Parameters()

expr = p.param.a.reactive() + p.param.b.reactive() + 3

expr

The resulting reactive expression now reflects the result of this operation and will update automatically when one of the inputs to the operation changes, e.g. if we update parameter `a`:

In [ ]:
p.a = 5

### Resolving the expression

To resolve the current value of the expression we can call `.rx.resolve()`:

In [ ]:
expr.rx.resolve()

### Using literal objects as inputs

Note that the input to a reactive expression can be anything, e.g. we can create a `reactive` object from a static, literal value, such as a string:

In [ ]:
string_template = param.reactive('Hello {name}!')

string_template

The `reactive` object now acts like a string so we can use the `.format` method to fill in the `string_template`:

In [ ]:
name = param.reactive('world')

str_expr = string_template.format(name=name)

str_expr

### Setting the input value

To update the input to a `reactive` object we can use the `.rx.set()` method:

In [ ]:
name.rx.set('there')

str_expr.rx.resolve()

### Special Methods & Limitations

A `reactive` proxy behaves like the underlying object it is wrapping, but only to the extent that Python allows. Certain operations cannot be implemented in this way, e.g. Python will not allow the `len` operation to return anything but a integer and the `is` statement always checks the immediate identity of its two operands. Therefore `reactive` implements certain operations as special methods that exist on the `.rx` namespace, these include:

- `.bool()`: Tests the truthiness of the object
- `.is_()`: Tests the object identity against another object.
- `.is_not()`: Tests the object identity against another object.
- `.len()`: The length of the expression
- `.pipe()`: Allows applying an arbitrary function with static or reactive arguments to the object.
- `.when()`: Generates a new expression that only updates when the provided dependency updates.
- `.where()`: Returns either the first or the second argument depending on the current value of the expression.

#### `.bool()`

Casts the current value to a boolean:

In [ ]:
param.reactive(1).rx.bool()

#### `.is_(arg)`

Checks the identity of the current value is the same as the argument to `.is_`

In [ ]:
param.reactive(None).rx.is_(None)

#### `.is_not(arg)`

Checks the identity of the current value is not the same as the argument to `.is_not`

In [ ]:
param.reactive(None).rx.is_not(None)

#### `.len()`

Returns the length of the object

In [ ]:
param.reactive([1, 2, 3]).rx.len()

#### `.pipe(func, *args, **kwargs)`

Pipes the current value into a function as the first argument, additional positional and keyword arguments can be passed:

In [ ]:
param.reactive(1).rx.pipe(add, 2)

#### `.when(*conditions)`

Useful when creating UIs to declare that the expression should only update when some other parameter changes, e.g. when a user clicks a button or triggers an expensive operation through some other mechanism.

Let us build an example, let's say we have some expensive function (here simulated using `time.sleep`). First we bind parameters `a` and `b` to this function and create a reactive expression from this function.

In [ ]:
import time

def expensive_function(a, b):
    time.sleep(2)
    return a * b

p = Parameters()

expensive_expr = param.bind(expensive_function, p.param.a, p.param.b).reactive()

The problem we face is that if we use this `expensive_expr` whenever `a` **or** `b` are changed then the expensive computation is triggered, i.e. if we want to change both before re-computing then we are forced to re-run the function twice. We therefore want to gate the computation behind a third variable, e.g. the `run` `Event` parameter on the `Parameters` class.

In [ ]:
gated_expr = expensive_expr.rx.when(p.param.run)

gated_expr

We can now safely change variables `a` and `b` without triggering the computation:

In [ ]:
p.a = 2
p.b = 4

gated_expr.rx.resolve()

But when we trigger the `run` parameter the expression will re-compute:

In [ ]:
p.param.trigger('run')

gated_expr.rx.resolve()

#### `.where(x, y)`

Where allows writing ternary conditions using the reactive paradigm. Ordinarily you can write something like this:

```python
value_a if some_condition else value_b
``` 

to return `value_a` or `value_b` depending on some condition. However since `if` statements are not re-evaluated when the condition changes we have to rewrite this case using `where`.

First we will declare a condition, here this is simply a `reactive` expression wrapping a boolean value:

In [ ]:
condition = param.reactive(True)

Now let's say we want to return either Parameter `a` or `b` depending on whether the condition is True or False. We can simply pass the values to `.where()`:

In [ ]:
p = Parameters(a=1, b=2)

ternary_expr = condition.rx.where(p.param.a, p.param.b)

ternary_expr

Since the initial value is `True` it returns the current value of `a`, which is `1`. However when we set the value to `False` it will return the value of `b`:


In [ ]:
condition.rx.set(False)

ternary_expr.rx.resolve()

Importantly, if we now change `b` the result will be reflected by the expression:

In [ ]:
p.b = 5

ternary_expr.rx.resolve()

While changes to `a` will no longer trigger any downstream updates until the condition is reversed again.